<a href="https://colab.research.google.com/github/trushnes/GTZAN-Genre-Classification/blob/main/MFCC_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Importing Libraries**

In [ ]:
# LIBRARIES
import json
import os
import math
import librosa

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


### **Feature Extraction**

**Extracting MFCC from all of the audio files**

In [ ]:
DATASET_PATH = '/content/drive/MyDrive/Data/genres_original'
JSON_PATH = "/content/drive/MyDrive/Data/data_10.json"
SAMPLE_RATE = 22050
TRACK_DURATION = 30 # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION

In [ ]:
def save_mfcc(dataset_path, json_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):

    data = {
        "mapping": [],
        "labels": [],
        "mfcc": [],
        }

    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        if dirpath is not dataset_path:
            semantic_label = dirpath.split("/")[-1]
            data["mapping"].append(semantic_label)
            print("\nProcessing: {}".format(semantic_label))

            
            for f in filenames:
                file_path = os.path.join(dirpath, f)
                if file_path != '/content/drive/MyDrive/Data_Reduced/jazz/jazz.00054.wav':
                signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
                
                
                    for d in range(num_segments):

                        start = samples_per_segment * d
                        finish = start + samples_per_segment
                        mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                        mfcc = mfcc.T

                        if len(mfcc) == num_mfcc_vectors_per_segment:
                            data["mfcc"].append(mfcc.tolist())
                            data["labels"].append(i-1)
                            print("{}, segment:{}".format(file_path, d+1))

    # saving MFCCs to json file in drive
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)

In [ ]:
# Runs Data Processing     
save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)

**Done :)**